In [10]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import vk
import numpy as np

Скачиваем с хтмл страницы первую тысячу человек

Собираем у них ссылки и ФИО

После отбираем уникальные фамилии и имена, чтобы искать больше людей

Веб-скрапим страницы и получаем ссылки и имена

In [11]:
with open('labeled name.html', 'r') as f:

    contents = f.read()

    soup = BeautifulSoup(contents, 'html.parser')

#     print(soup.prettify())

In [13]:
df = pd.DataFrame(columns = ['link', 'id', 'name', 'last_name', 'city', 'education', 'interests', 'music', 'movies', 'tv', 'books', 'games', 'can_see_audio'])

info = soup.find_all('div', class_ = 'people_row search_row clear_fix')

for item in info:
    div = item.find_all('div', class_ = 'labeled name')
    a_class = div[0].find_all('a')
#     print(i)
#     print(type(a_class[0]))
    link = a_class[0].get('href')
    spl = a_class[0].text.split(' ')
    name = spl[0]
    last_name = spl[1]
#     name = str(a_class[0].find_all('img')[0]).split("\"")[1]
    
    test = item.find_all('div', class_ = 'labeled')
    if (len(test) > 1):
        if ('Россия' in test[1].text):
            city = test[1].text
        else:
            city = None
    else:
        city = None
    df = df.append({'link': link, 'name': name, 'last_name': last_name, 'city': city}, ignore_index=True)
    
df.to_excel('init_dataset.xlsx')

In [15]:

print(len(np.unique(df['name'])))

print(len(np.unique(df['last_name'])))

391
870


In [19]:
d = {'name': np.unique(df['name'])}
f = {'name': np.unique(df['last_name'])}
unique_names = pd.DataFrame(data = d)
unique_lasts = pd.DataFrame(data = f)
# unique_names = unique_names.append(np.unique(df['last_name']))
unique_values = pd.concat([unique_names, unique_lasts], ignore_index=True)
unique_values.to_excel('unique_names.xlsx')

1. нужно каждый день среди этих имен и фамилий майнить хтмл страницы

2. Каждый день по 75 страниц таких сохранять.

3. Сохранять из разряда 1 2 3 4 и тд, чтобы после можно было замайнить все данные в эксель

4. В экселе сразу разделить имена и фамилии, сделать раздельно

5. Параллельно майнить по людям данные из фссп. Можно по 100 людей в час, 1000 в день

6. Автоматизировать процесс 

Обрабатываем ссылки через вк апи, забираем всю возможно важную информацию с минимальным количеством вызовом апи

In [ ]:
dataset = pd.DataFrame(columns = ['link', 'name', 'id', 'bday', 'home_town' 'sex', 'and so on' ])
# https://vk.com/dev/users.get

In [ ]:
class ConfigAPI(object):
    __token = None
    api = None

    #Стартовый обработчик. Подключает TOKEN VK API к приложению
    def init(self):
        print('Вставьте ссылку в строку браузера для получения токена VK API. ')
        print('https://oauth.vk.com/authorize?client_id=7305133&display=page&scope=status,video,photos,friends,wall,notes,groups,docs&response_type=token&v=5.103&state=123456')
        print('После скопируйте ссылку из браузера и вставьте сюда')
        val = str(input())
        self.__get_token_by_link(val) #токен обрабатывается на ошибки в этом методе
        self.__vk_api_on() # подключение VK API к системе


    def __get_token_by_link(self, str):
        arr = str.split('=')
        if(len(arr) >= 2):
            arr2 = arr[1].split('&')
            token = arr2[0]
            print(token)
            self.__token = token
        else:
            print('ссылка повреждена, попробуйте еще раз!')
            print(' ')
            self.init()

    def __vk_api_on(self):
        token = self.__token
        try:
            if(token != None):
                session = vk.Session(token)
                self.api = vk.API(session, v=5.103)

                user = self.api.users.get(user_ids=1) #не работает...

                print('Токен подключен!')
                print(' ')
            else:
                print('Токен отсутствует, попробуйте еще раз!')
                print(' ')
                self.init()
        except Exception:
            print('Токен недействителен или поврежден, попробуйте еще раз')
            print(' ')
            self.init()

In [ ]:
c = ConfigAPI()
c.init()

In [ ]:
people = c.api.users.get(user_ids = [], fields = ['sex', 'bdate', 'city', 'home_town', 'education', 'universities', 'schools', 'status', 'last_seen', 'followers_count', 'common_count', 'occupation', 'nickname', 'relatives', 'relation', 'personal', 'connections', 'exports', 'activities', 'interests', 'music', 'movies', 'tv', 'books', 'games', 'about', 'quotes', 'can_see_audio'])

Прогружаем и скачиваем все новые страницы для полного поиска(не работает, прокрутка и скачивание в ручную

Прокрутка:

https://coderoad.ru/20986631/Как-я-могу-прокрутить-веб-страницу-используя-selenium-webdriver-в-python

Сохранение хтмл:

https://stackoverflow.com/questions/42900214/how-to-download-a-html-webpage-using-selenium-with-python

Перерабатываем города в регионы через вк апи(не нужно, если искать только по РТ)

1) Загоняем людей из таблицы (ФИО, регион) в запрос жсон, у которых стоит не обработанные запросы(1) или запросов нет (обработанный запрос(0))

2) Отправляем жсон на обработку. Когда обработка пошла, этим людям ставим 1. Сохраняем в отдельную таблицу id жсон, индексы начало/конец, токен, статус

3) Ждем/проверяем обработку

4) Если обработался запрос и получены данные, то людям ставим 0 в таблице, сохраняем данные, обновляем статус в таблице жсона или удаляем жсон

    Если не обработался запрос(ошибка/ограничения и тд), то запускаем позже или выясняем причину.
    Сохраняем ошибки в логи id жсона, ошибка

In [ ]:
class Name(object):
    name = None
    second_name = None
    def __init__(self, name, second_name):
        self.name = name
        self.second_name = second_name

In [ ]:
def get_fssp(Name, region):
    

In [ ]:
class DataDownloader(object):
    apiUrl = None
    token = None
    token1 = None
    is_server_limit = None
    code_region_db = None
    current_region = None

    def __init__(self):
        self.apiUrl = "https://api-ip.fssprus.ru/api/v1.0/"
        self.token = self.get_token()
#         self.token1 = self.get_token1()
        self.code_region_db = self.__init_code_region()

#     def __init_code_region(self):
#         db = pd.read_csv('KOD_region.csv')
#         return db

    def get_token(self):
        tok = open('token.txt')
        token = tok.read()
        token = token.split('\n')
        return token[0]

#     def get_token1(self):
#         tok = open('token1.txt')
#         token = tok.read()
#         token = token.split('\n')
#         return token[0]

    def to_string(self, person):
        result = "&region={}&firstname={}&lastname={}&secondname={}&birthday={}".format(person.region, person.firstName,
                                                                                        person.lastName,
                                                                                        person.secondName,
                                                                                        person.birthday)
        return result

    def GetTaskState(self, task):
        comStr = 'status?token={}&task={}'.format(self.token, task)
        resp = self.GetTaskCommand(comStr)
        print(' ')
        print(resp)
        print(' ')
        if (resp.get('code') != None):
            self.is_server_limit = resp['code']

        return resp['response']['status']

    def GetTaskCommand(self, comStr):
        line = self.apiUrl + comStr
        #     print(line)
        r = requests.get(line)
        return r.json()

    def GetResult(self, task):
        comStr = 'result?token={}&task={}'.format(self.token, task)
        line = self.apiUrl + comStr
        #     print(line)
        r = requests.get(line)
        if (r.json()['code'] == 429):
            self.is_server_limit = 429
        return r

    def add_to_db(self, data):
        # если папка существует
        if (os.path.exists('db')):
            print('Database exists')
            #     зайти в папку
            os.chdir('db')
            #     проверить файл в папке
            if not os.path.isfile('dataset.xlsx'):

                #         создать файл
                db = pd.DataFrame(
                    columns=['name', 'lastname', 'secondname', 'birthday', 'hometown', 'number_ip', 'credit', 'details',
                             'department', 'status'])

                db = pd.concat([db, data])
                db.to_excel('dataset.xlsx')
            else:
                db = pd.read_excel('dataset.xlsx')
                db = pd.concat([db, data])
                db.to_excel('dataset.xlsx')

            db = pd.read_excel('dataset.xlsx')
            db = db[db.columns.drop(list(db.filter(regex='^Unnamed')))]
            db.to_excel('dataset.xlsx')

            os.chdir('..')
            return db
        # если не существует
        else:
            print('Database doesn\'t exist')
            os.makedirs('db')
            os.chdir('db')
            db = pd.DataFrame(
                columns=['name', 'lastname', 'secondname', 'birthday', 'hometown', 'number_ip', 'credit', 'details',
                         'department', 'status'])
            db = pd.concat([db, data])
            #         db = db.loc[:, ~db.columns.str.contains('^Unnamed')]
            #         db.to_excel('dataset.xlsx')
            #         db = pd.read_excel('dataset.xlsx')
            db = db[db.columns.drop(list(db.filter(regex='^Unnamed')))]
            db.to_excel('dataset.xlsx')
            os.chdir('..')
            return db

    def new_init(self):
        warnings.filterwarnings('ignore')
        df = self.load_tables()

        for i in range(len(df)):
            print(" ")
            print("Обрабатывается {}й человек".format(i))
            if (len(df[df['СТАТУС'] != 0]) > 0):
                if (df['СТАТУС'][i] != 0):
                    self.check_req_limits()
                    if (self.is_server_limit == 429):
                        return self.is_server_limit

                    person = self.create_person(df['ФИО'][i], df['ДАТА'][i])
                    if (person != None):
                        code = self.get_person_info(person)
                        if (code == 429 or code == 430):
                            df = df[df['СТАТУС'] != 0]
                            df.to_excel('input.xlsx')
                            return self.is_server_limit
                        elif (code == 0):
                            df['СТАТУС'][i] = 0
                            df.to_excel('input.xlsx')
            else: break

        df.to_excel('input.xlsx')

        return self.is_server_limit

    def get_response(self, js):
        url = self.apiUrl + 'search/group'
        data = json.dumps(js)
        headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
        r = requests.post(url, data= data, headers=headers)
        return r.json()

    def create_person(self, fio, date):
        fio_split = fio.split(' ')
        if len(fio_split) == 3:
            first_name = fio_split[1]
            second_name = fio_split[0]
            last_name = fio_split[2]
        else:
            print('Некорректно введенное ФИО {}'.format(fio))
            return None

        is_letter_contains = re.search('[a-zA-Zа-яА-Я]', date)
        if (is_letter_contains == None):
            date = self.create_date(date)
        else:
            print('Некорректно введенная дата рождения {}'.format(date))
            return None

        person = Person(first_name, second_name, last_name, date)
        return person

    def create_date(self, date):
        datespl = date.split(".")
        if (len(datespl) > 3):
            date = datespl[2] + '.' + datespl[1] + '.' + datespl[0]
        return date

    def get_person_info(self, person):
        js1, js2 = self.get_json(person)

        self.get_info(js1)
        if (self.is_server_limit == 429):
            if (js1['token'] == self.token):
                js1['token'] == self.token1
                self.get_info(js1)
            elif (js1['token'] == self.token):
                js1['token'] == self.token
                self.get_info(js1)
            else:
                print('сервер не отвечает, перезапуск через минуту')
                time.sleep(60)
                js1['token'] == self.token1
                self.get_info(js1)
            # return self.is_server_limit

        if (self.is_server_limit == 429):
            print('Программа ожидает снятия ограничений сервера')


        self.get_info(js2)
        if (self.is_server_limit == 429):
            if (js2['token'] == self.token):
                js2['token'] == self.token1
                print('Меняем токен')
                self.get_info(js2)
            elif (js2['token'] == self.token):
                js2['token'] == self.token
                print('Меняем токен')
                self.get_info(js2)
            else:
                print('сервер не отвечает, перезапуск через минуту')
                time.sleep(60)
                js2['token'] == self.token1
                self.get_info(js2)
        return self.is_server_limit

    def get_info(self, js):
        r = self.get_response(js)
        print(' ')
        print(r)
        print(' ')
        if (self.get_resp_status(r['code'])):
            if (r['code'] == 0):
                task = r['response']['task']
                status = self.GetTaskState(task)
                if (self.is_server_limit == 429):
                    return self.is_server_limit

                start_time = time.time()
                while (status != 0):
                    time.sleep(3)
                    status = self.GetTaskState(task)

                    if (self.is_server_limit == 429):
                        return self.is_server_limit

            if (self.get_result(task)):
                return 0

            else: return self.is_server_limit

    def get_result(self, task):
        result = self.GetResult(task)
        if (self.is_server_limit == 429):
            return False

#         ts = pd.read_excel('timing.xlsx')
#         el_ind = len(ts)
#         req = result.json()['status']
#         ts.loc[el_ind, 'request'] = req
#         ts.loc[el_ind, 'time'] = datetime.datetime.now()
#         ts = ts[ts.columns.drop(list(ts.filter(regex='^Unnamed')))]
#         ts.to_excel('timing.xlsx')
#         print('Отработало')

        dt = pd.DataFrame(
            columns=['name', 'lastname', 'secondname', 'birthday', 'hometown', 'number_ip', 'credit', 'details',
                     'department', 'status'])

        resp_count = 0
        #     вытаскиваем жсон
        length = len(result.json()['response']['result'])
        for i in range(length):
            res = result.json()['response']['result'][i]['result']

            length_res = len(res)
            if (length_res) > 0:
                resp_count += 1
            for j in range(0, length_res):
                el = len(dt)
                spl = res[j]['name'].split()
                dt.loc[el, 'name'] = spl[1]
                dt.loc[el, 'lastname'] = spl[0]
                dt.loc[el, 'secondname'] = spl[2]
                dt.loc[el, 'birthday'] = spl[3]
                s = ''
                len_s = len(spl)
                for k in range(4, len_s):
                    s += ' ' + spl[k]
                dt.loc[el, 'hometown'] = s
                dt.loc[el, 'number_ip'] = res[j]['exe_production']
                dt.loc[el, 'credit'] = res[j]['subject']
                dt.loc[el, 'details'] = res[j]['details']
                dt.loc[el, 'department'] = res[j]['department']
                dt.loc[el, 'status'] = res[j]['ip_end']


        #     добавляем обработанного челика в общую бд
        db = self.add_to_db(dt)

        return True

    def get_resp_status(self, code):
        if (code == 401):
            print('invalid token')
            self.is_server_limit = code
        elif (code == 400):
            print("invalid request params. Code 400")
            self.is_server_limit = code
        elif (code == 429):
            print('Перезапустите программу. Code 429: чрезмерное количество запросов')
            self.is_server_limit = code
        else: return True
        return False

    def get_json(self, person):
        sqr = {
            "token": self.token,
            "request": []}

        for i in range(0, 50):
            item = {"type": 1,
                    "params": {"firstname": person.firstName, "lastname": person.lastName,
                               "secondname": person.secondName, "region": int(self.code_region_db['KOD_region'][i]),
                               "birthdate": person.birthday
                               }}
            sqr['request'].append(item)

        sqr1 = {
            "token": self.token,
            "request": []}

        length_db = len(self.code_region_db)

        for j in range(50, length_db):
            item = {"type": 1,
                    "params": {"firstname": person.firstName, "lastname": person.lastName,
                               "secondname": person.secondName, "region": int(self.code_region_db['KOD_region'][j]),
                               "birthdate": person.birthday
                               }}
            sqr1['request'].append(item)

        return sqr1, sqr

    def check_req_limits(self):
        ts = pd.read_excel('timing.xlsx')
        ts = ts[ts.columns.drop(list(ts.filter(regex='^Unnamed')))]

        сount_hour = \
        ts[(ts['time'] > (datetime.datetime.now() - datetime.timedelta(hours=1))) & (ts['request'] == 'success')][
            'time'].count()
        count_day = \
        ts[(ts['time'] > (datetime.datetime.now() - datetime.timedelta(days=1))) & (ts['request'] == 'success')][
            'time'].count()

        if сount_hour > 100 or count_day > 1000:
            self.is_server_limit = 429

    def load_tables(self):
        df = pd.read_excel('input.xlsx')
        df = df[df.columns.drop(list(df.filter(regex='^Unnamed')))]
        df['ДАТА'] = df['ДАТА'].astype(str)
        len_df = len(df)
        for i in range(len_df):
            df['ДАТА'][i] = df['ДАТА'][i].replace('-', '.')
            df['ДАТА'][i] = df['ДАТА'][i].replace(' ', '.')

        return df

    def recreate_database(self):
        input_db = pd.DataFrame(columns=['ФИО', 'ДАТА', 'СТАТУС'])
        input_db.to_excel('input.xlsx')

        timing_db = pd.DataFrame(columns=['request', 'time', 'resp_count'])
        timing_db.to_excel('timing.xlsx')

        with open("token.txt", "w") as file:
            file.write("")

In [ ]:
d = DataDownloader()
per = Person('МАРИАННА', 'АРМЕНАКОВНА', 'АВАКЯН', '20.02.1975')
# js1, js2 = d.get_json(per)
# r = d.get_info(js1)
t = d.get_person_info(per)
t

In [ ]:
p = d.GetResult(k)
p.json()

Посмотреть работает ли на одиночном запросе

Посмотреть как работает структура, переделать под сохранение токенов в таблицу и и проверки токенов и их обновление
